In [1]:
import pandas as pd
import urllib
import time
import json

In [37]:
studentBehavior = pd.read_json("studentBehaviorInfoOver40Class_1213.json", orient="index")
csv_file = pd.read_csv("clusteringData.csv",header=None)

In [34]:
# print(studentBehavior[0])
# print studentBehavior[0]["memberId"]
# print (studentBehavior[0]["chosenVideo"])
# print (studentBehavior[0]["vocabularyList"])
print(video_to_cluster)

## Desired json structure
# [
#     {
#         "memberId": 12345,
#         "videoSequence":
#             [
#                 3980,
#                 7680,
#                 1235
#             ],
#         "skippingRatioSequence":
#             [
#                 0,
#                 0,
#                 0.3
#             ],
#         "scoreSequence":
#             [
#                 80,
#                 70,
#                 75
#             ],
#         "dictionarySequence":
#             [
#                 0,
#                 0,
#                 0
#             ],
#         "skippingIndex": 10,
#         "clusterSequence":
#             [
#                 "C1",
#                 "C1",
#                 "C2"
#             ]
        
#     }
# ]



13354


In [42]:
# Generate Users Profile File

# This method return a mapping of video(key) 
# and and array of scores(value)
def parse_video_scores(scores_object):
    result = {}
    for score in scores_object:
        if score['postId'] not in result:
            result[score['postId']] = []
        result[score['postId']].append(score['score'])
    return result
            
def parse_video_words(words_object):
    result = {}
    for word in words_object:
        if word['postId'] not in result:
            result[word['postId']] = []
        result[word['postId']].append(word['word'])
    return result

def parse_csv_file(csv_file):
    result = {}
    for index in range(1,len(csv_file[0])):
        result[csv_file[0][index]] = int(csv_file[1][index])
    return result
    

def calc_skipping_ratio(scores):
    countSkipped = 0
    for score in scores:
        if score < 0:
            countSkipped += 1
    return round(countSkipped*1.0/len(scores)*1.0,2)

def calc_avg_score(scores):
    countSkipped = 0
    sumScores = 0
    for score in scores:
        if score < 0:
            countSkipped += 1
        else:
            sumScores += score
                
    if len(scores)-countSkipped == 0:
        return 0
    else:
        return round(sumScores*1.0/(len(scores)*1.0-countSkipped*1.0),2)
    
lenStudentBehavior = len(pd.read_json("studentBehaviorInfoOver40Class_1213.json"))
usersProfilesJsonWithIndexes = {}
video_to_cluster = parse_csv_file(csv_file)
for index in range(lenStudentBehavior):
    memberId = studentBehavior[index]["memberId"]
    if memberId not in usersProfilesJsonWithIndexes:
        studentProfile                          = {}
        studentProfile["memberId"]              = memberId
        studentProfile["videoSequence"]         = []
        studentProfile["skippingRatioSequence"] = []
        studentProfile["avgScoreSequence"]         = []
        studentProfile["dictionarySequence"]    = []
        studentProfile["skippingIndex"]         = -1
        studentProfile["clusterSequence"]       = []
        usersProfilesJsonWithIndexes[memberId] = studentProfile
    
    # Addid values for every key
    # Video Sequence
    usersProfilesJsonWithIndexes[memberId]["videoSequence"].extend(studentBehavior[index]["chosenVideo"])
    
    
    videos_scores = parse_video_scores(studentBehavior[index]["listenScore"])
    video_words = parse_video_words(studentBehavior[index]["vocabularyList"])
    
    
    # SkippingRationSequence
    for video in studentBehavior[index]["chosenVideo"]:
        scores = videos_scores[video]
        usersProfilesJsonWithIndexes[memberId]["skippingRatioSequence"].append(calc_skipping_ratio(scores))
        # Average Score Sequence
        avg = calc_avg_score(scores)
        usersProfilesJsonWithIndexes[memberId]["avgScoreSequence"].append(avg)
        
        # DictionarySequence
        if str(video) in video_words:
            words = video_words[str(video)]
            usersProfilesJsonWithIndexes[memberId]["dictionarySequence"].append(len(words))
        else:
            usersProfilesJsonWithIndexes[memberId]["dictionarySequence"].append(0)
            
        # ClusterSequece
        usersProfilesJsonWithIndexes[memberId]["clusterSequence"].append(video_to_cluster[str(video)])
            
        
    # skippingIndex
    skip_index = [ n for n,i in enumerate(usersProfilesJsonWithIndexes[memberId]["skippingRatioSequence"]) if i>0.0 ][0]
    usersProfilesJsonWithIndexes[memberId]["skippingIndex"] = skip_index
        
    
    
print(usersProfilesJsonWithIndexes['118065'])
fileStudentProfile = open("studentProfile.json", "w")
fileStudentProfile.write("[")
lenObjects = len(usersProfilesJsonWithIndexes)
it = 0
for key, value in usersProfilesJsonWithIndexes.items():
    json.dump(value,fileStudentProfile)
    if it != (lenObjects-1):
        fileStudentProfile.write(", ")
    it = it + 1
    


fileStudentProfile.write("]")
fileStudentProfile.close()

KeyError: 118065

# Notes

1. We have three different skipRatio behavior
    User that goes from 0 to 1
    Users that goes from 0 to [a range gratter than 0] and stop
    Users that goes from 0 to [a range gratter than 0] and then skippRation 1

2. The skipIndex shows the index where the skipRation is gratter than 0
    
